# TRYING/TESTING necessary libraries for document processing #


## STEP 1: Reading and extracting texts from various document formats ##

In [5]:
from pathlib import Path
file_path = Path("data/raw_documents/receipts/train_ticket_receipt.pdf").resolve()
print("Resolved Path:", file_path)


Resolved Path: C:\Users\User\Documents\SideProject\personal_doc_chatbot_with_RAG\data\raw_documents\receipts\train_ticket_receipt.pdf


#### Extract from PDFs ####


In [4]:
# Reading and extracting texts from PDFs
# by using PyMuPDF
import pymupdf4llm

#use ticket_train receipt as an example
md_text = pymupdf4llm.to_markdown("c:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/receipts/train_ticket_receipt.pdf")

print(md_text)




Processing c:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/receipts/train_ticket_receipt.pdf...
[                                        ] (0/2===================[====================                    ] (1/2===================[========================================] (2/2]
# TAPAH ROAD - ANAK BUKIT


### Berlepas / Departure:


### Tiba / Arrival:


### 16/11/2023 12:46 PM 16/11/2023 03:57 PM

 MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult

 xxxxxxxx0849


#### MYR 46.00/DebitCard

 K231189695108 ** T231175999163 (Single)


#### Online


Airasia Ride Promo Code : KTMRIDE
Sila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.

Tertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.

KTMB CALL CENTER +603-2267 1200


### 9420

 EG9420

###### Coach/Seat

## C/5C

###### Gold


-----

-----




In [ ]:
#Save the extracted text into .md format file



### Extract from scanned PDFs/images

In [5]:
# Reading and extracting texts from scanned PDFs
# By using easyOCR (en,ms)
import easyocr
from pdf2image import convert_from_path
reader = easyocr.Reader(['en','ms'])


In [6]:

poppler_path = r"C:/Users/User/poppler-24.08.0/Library/bin"
pdf_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/personal_documents/spm_certificate.pdf"
import numpy as np
images = convert_from_path(pdf_path,poppler_path=poppler_path)

all_text = []
# Process each page image directly
for i, img in enumerate(images):
    # Convert PIL image to NumPy array
    img_np = np.array(img)

    #read the text from image
    text = reader.readtext(img_np, detail=0) #extracts the text 
    page_text = "\n".join(text) #joins the text lines
    all_text.append(f"## Page{i+1}\n{page_text}") #format each page by including a title/number of pages

markdown_text = "\n\n".join(all_text)
print(markdown_text)
    


## Page1
KEMENTERIAN PENDIDIKAN MALAYSIA
MINISTRY OF EDUCATION MALAYSIA
LEMBAGA PEPERIKSAAN
EXAMINATIONS SYNDICATE
Calon yang namanya tercatat di bawah telah menduduki peperiksaan Sijil Pelajaran Malaysia (SPM) dan
layak dianugerahi
SIJIL PELAJARAN MALAYSIA
MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR
020630-02-0849
KC208A089
SMK AGAMA KEDAH
Mata Pelajaran
Gred
Subject
Grade
BAHASA MELAYU
A
(CEMERLANG TINGGI)
BAHASA INGGERIS
A
(CEMERLANG TINGGI)
PENDIDIKAN ISLAM
A+
(CEMERLANG TERTINGGI)
SEJARAH
A+
(CEMERLANG TERTINGGI)
MATHEMATICS
A
(CEMERLANG TINGGI)
BAHASA ARAB
B+ (KEPUJIAN TERTINGGI)
ADDITIONAL MATHEMATICS
C
(KEPUJIAN)
PHYSICS
A-
(CEMERLANG)
CHEMISTRY
A
(CEMERLANG TINGGI)
BIOLOGY
A+
(CEMERLANG TERTINGGI)
HIFZ AL-QURAN
B+ (KEPUJIAN TERTINGGI)
MAHARAT AL-QURAN
B+ (KEPUJIAN TERTINGGI)
JUMLAH MATA PELAJARAN DUA BELAS
PEPERIKSAAN TAHUN 2019
Pengarah Peperiksaan
Director of Examinations
191258556
Kementerian Pendidikan Malaysia
A
04845697
Ministry of Education Malaysia
2423
iaur
SebTaheh MuTu

In [ ]:
# Save the extracted text into .md format

In [ ]:
# Reading and extracting texts from .docx/.docs



## STEP 2: Chunking the extracted text

### Implement hybrid chunking (Split based on the headings of the markdown + fixed-length split )


In [3]:
# use the md_text as an example

from markdown_it import MarkdownIt

md = MarkdownIt() #initialize the library
print(md.render(md_text))

<h1>TAPAH ROAD - ANAK BUKIT</h1>
<h3>Berlepas / Departure:</h3>
<h3>Tiba / Arrival:</h3>
<h3>16/11/2023 12:46 PM 16/11/2023 03:57 PM</h3>
<p>MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult</p>
<p>xxxxxxxx0849</p>
<h4>MYR 46.00/DebitCard</h4>
<p>K231189695108 ** T231175999163 (Single)</p>
<h4>Online</h4>
<p>Airasia Ride Promo Code : KTMRIDE
Sila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.</p>
<p>Tertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.</p>
<p>KTMB CALL CENTER +603-2267 1200</p>
<h3>9420</h3>
<p>EG9420</p>
<h6>Coach/Seat</h6>
<h2>C/5C</h2>
<h6>Gold</h6>
<hr />
<hr />



In [7]:
from collections import defaultdict
from pathlib import Path

def split_markdown_by_headings(md_text):
    md = MarkdownIt()  # Initialize Markdown parser
    tokens = md.parse(md_text)  # Parse Markdown
    sections = defaultdict(list)  # Store content based on headings / split content based on headings
    
    current_heading = "No Heading"  # Default if no heading at start

    for token in tokens:
        if token.type == "heading_open":  # Detect heading (h1, h2, h3, etc.)
            level = int(token.tag[1])  # Extract heading level (1, 2, 3)
            next_token = tokens[tokens.index(token) + 1]  # Get heading text
            if next_token.type == "inline":
                current_heading = f"{'#' * level} {next_token.content}"  # Format heading
        
        elif token.type == "paragraph_open":  # Detect paragraphs
            next_token = tokens[tokens.index(token) + 1]
            if next_token.type == "inline":
                sections[current_heading].append(next_token.content)  # Store content under heading

    # Convert dictionary values to strings
    return {key: "\n".join(value) for key, value in sections.items()}


split_sections = split_markdown_by_headings(md_text)
print(split_sections)



{'### 16/11/2023 12:46 PM 16/11/2023 03:57 PM': 'MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849', '#### MYR 46.00/DebitCard': 'K231189695108 ** T231175999163 (Single)', '#### Online': 'Airasia Ride Promo Code : KTMRIDE\nSila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.\nTertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.\nKTMB CALL CENTER +603-2267 1200', '### 9420': 'EG9420'}


In [8]:
import nltk

# for each sections, check if the tokens in a section is more than 512 tokens, divide the section into more subsetions by using NLTK

# function to chunk the text if it has more than 512 tokens
def chunk_text(text, max_tokens=512):
    words = nltk.word_tokenize(text)
    chunks = []

    for i in range(0, len(words), max_tokens):
        chunk_words = words[i: i + max_tokens]
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
    return chunks
    
new_sections = {}


for heading, content in split_sections.items():
    tokens = nltk.word_tokenize(content)  # Count tokens in the content
    
    if len(tokens) > 512:
        print(f"Chunking '{heading}' as it has {len(tokens)} tokens...")
        chunked_subsections = chunk_text(content)  # Split content into 512-token chunks
        
        # Store each chunk under new subsections
        for i, chunk in enumerate(chunked_subsections, start=1):
            new_heading = f"{heading} (Part {i})"  # Create subheading
            new_sections[new_heading] = chunk
    else:
        new_sections[heading] = content  # Keep original section
    

    print(new_sections.items())


dict_items([('### 16/11/2023 12:46 PM 16/11/2023 03:57 PM', 'MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849')])
dict_items([('### 16/11/2023 12:46 PM 16/11/2023 03:57 PM', 'MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849'), ('#### MYR 46.00/DebitCard', 'K231189695108 ** T231175999163 (Single)')])
dict_items([('### 16/11/2023 12:46 PM 16/11/2023 03:57 PM', 'MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849'), ('#### MYR 46.00/DebitCard', 'K231189695108 ** T231175999163 (Single)'), ('#### Online', 'Airasia Ride Promo Code : KTMRIDE\nSila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.\nTertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.\nKTMB CALL CENTER +603-2267 1200')])
dict_items([('### 16/11/2023 12:46 PM 16/11/2023 03:57 PM', 'MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849'), ('#### MYR 46.00/DebitCard', 'K231189695108 ** T2311

## Step 3: Save the chunked data into a json format file

In [17]:
import json
import nltk
from nltk.tokenize import word_tokenize
from pathlib import Path

chunk_counter =1 
source = "train_ticket_receipt"
# Path to the existing JSON file
output_path = "processed_data.json"

# Load existing data if file exists, otherwise start with an empty list
if Path(output_path).exists():
    with open(output_path, "r", encoding="utf-8") as file:
        try:
            existing_data = json.load(file)
            if not isinstance(existing_data, list):  # Ensure it's a list
                existing_data = []
        except json.JSONDecodeError:  # Handle empty or corrupted JSON
            existing_data = []
else:
    existing_data = []

# Convert new chunks to JSON format with token count
new_json_data = []
for heading, content in new_sections.items():
    token_count = len(word_tokenize(content))  # Count tokens
    new_json_data.append({
        "chunk_id": chunk_counter,
        "heading": heading,
        "content": content,
        "source": source,
        "tokens": token_count
    })
    chunk_counter=chunk_counter+1

# Append new data to existing data
existing_data.extend(new_json_data)

# Save the updated JSON back to the file
with open(output_path, "w", encoding="utf-8") as file:
    json.dump(existing_data, file, indent=4, ensure_ascii=False)

print(f"New document chunks appended to {output_path}")


New document chunks appended to processed_data.json
